Connected to Python 3.13.7

# Libraries

In [84]:
import pandas as pd
import numpy as np
import json

WAREHOUSE_COORDS = (-73.985428, 40.748817)  # Example: Empire State Building coordinates

RANDOM_STATE = 2000
N_STATIONS = 19
output_dirs = ['../processed_data/distance_matrix_small.csv',
                '../processed_data/duration_matrix_small.csv',
                '../processed_data/some_stations_small.csv']

# Read the Bici Data

In [85]:
final_data = pd.DataFrame()

for file in ["../in_data/202511-citibike-tripdata_1.csv",
             "../in_data/202511-citibike-tripdata_2.csv",
             "../in_data/202511-citibike-tripdata_3.csv",
             "../in_data/202511-citibike-tripdata_4.csv"]:
    df = pd.read_csv(file)
    final_data = pd.concat([final_data, df], ignore_index=True)

final_data.head()

C:\Users\matte\AppData\Local\Temp\ipykernel_18248\3652892453.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\matte\AppData\Local\Temp\ipykernel_18248\3652892453.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\matte\AppData\Local\Temp\ipykernel_18248\3652892453.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\matte\AppData\Local\Temp\ipykernel_18248\3652892453.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CEE2FCDE8E0BC6F6,electric_bike,2025-11-11 10:08:01.447,2025-11-11 10:22:35.426,West Thames St,5114.06,Thompson St & Bleecker St,5721.07,40.708347,-74.017134,40.728401,-73.999688,member
1,1C6390981F71FC79,electric_bike,2025-11-09 05:08:42.937,2025-11-09 06:13:39.013,8 Ave & W 38 St,6526.05,8 Ave & W 33 St,6450.12,40.754610,-73.991770,40.751568,-73.993769,casual
2,5D13C1A39F5DF17C,electric_bike,2025-11-14 13:12:43.707,2025-11-14 13:20:29.147,W 54 St & 9 Ave,6920.05,8 Ave & W 33 St,6450.12,40.766040,-73.987370,40.751568,-73.993769,member
3,71CCA5EF65E27F1B,classic_bike,2025-11-12 17:27:45.060,2025-11-12 17:46:34.303,West End Ave & W 60 St,7059.08,W 82 St & Central Park W,7304.08,40.772370,-73.990050,40.782750,-73.971370,member
4,974E05C0A1E4A2DA,electric_bike,2025-11-11 11:45:05.368,2025-11-11 11:54:55.138,Morton St & Greenwich St,5772.05,8 Ave & W 33 St,6450.12,40.731150,-74.008870,40.751568,-73.993769,member


# Filter the data by time #

In [86]:
biciData = final_data.copy()
biciData['started_at'] = pd.to_datetime(biciData['started_at'])
biciData['ended_at'] = pd.to_datetime(biciData['ended_at'])

biciData['start_station_id'] = biciData['start_station_id'].astype(str).str.strip()
biciData['end_station_id'] = biciData['end_station_id'].astype(str).str.strip()

# Filter only the data for November 2025 and during weekdays and 7AM - 9AM
biciData = biciData[(biciData['started_at'].dt.month == 11) &
                    (biciData['started_at'].dt.year == 2025) &
                    (biciData['started_at'].dt.dayofweek < 5) &  # Monday to Friday
                    (biciData['started_at'].dt.hour >= 7)
                    & (biciData['started_at'].dt.hour < 9)]

biciData.reset_index(drop=True, inplace=True)
print(biciData.shape)

(325724, 13)


In [87]:
biciData.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,B9ACE7CC15728711,electric_bike,2025-11-05 07:05:08.387,2025-11-05 07:10:16.112,E 33 St & 5 Ave,6322.01,8 Ave & W 33 St,6450.12,40.747659,-73.984907,40.751568,-73.993769,member
1,58EDC7AF1E1AE185,electric_bike,2025-11-12 07:03:11.421,2025-11-12 07:08:07.863,9 Ave & W 45 St,6717.06,8 Ave & W 33 St,6450.12,40.760193,-73.991255,40.751568,-73.993769,member
2,F749CA8F4232897D,electric_bike,2025-11-11 08:51:36.729,2025-11-11 09:15:16.397,Evergreen Ave & Noll St,4873.08,Thompson St & Bleecker St,5721.07,40.701060,-73.933180,40.728401,-73.999688,member
3,A81A5F8B1FD8F11A,classic_bike,2025-11-05 07:37:02.232,2025-11-05 07:47:57.711,Washington Pl & 6 Ave,5838.09,8 Ave & W 33 St,6450.12,40.732241,-74.000264,40.751568,-73.993769,casual
4,8D5706EFABFCDF51,electric_bike,2025-11-05 07:22:02.710,2025-11-05 07:27:03.100,Madison Ave & E 82 St,7188.13,W 82 St & Central Park W,7304.08,40.778131,-73.960694,40.782750,-73.971370,member


# Collect all the unique stations

In [88]:
stations = pd.DataFrame(columns=['station_name', 'station_id', 'latitude', 'longitude'])

stations['station_id'] = pd.concat([biciData['start_station_id'], biciData['end_station_id']]).unique()
stations['station_name'] = stations['station_id'].map(biciData.drop_duplicates('start_station_id').set_index('start_station_id')['start_station_name']).fillna(
    stations['station_id'].map(biciData.drop_duplicates('end_station_id').set_index('end_station_id')['end_station_name'])
)

# Ensure unique index for mapping by dropping duplicates and setting index
start_lat_map = biciData.drop_duplicates('start_station_id').set_index('start_station_id')['start_lat']
end_lat_map = biciData.drop_duplicates('end_station_id').set_index('end_station_id')['end_lat']

stations['latitude'] = stations['station_id'].map(start_lat_map).fillna(
    stations['station_id'].map(end_lat_map)
)
stations['longitude'] = stations['station_id'].map(biciData.drop_duplicates('start_station_id').set_index('start_station_id')['start_lng']).fillna(
    stations['station_id'].map(biciData.drop_duplicates('end_station_id').set_index('end_station_id')['end_lng'])
)

print(f"The number of unique stations is: {stations.shape[0]}")

The number of unique stations is: 2217


Given the API has a limit of 100 coordinate calls

In [89]:

some_stations = stations.sample(n=N_STATIONS, random_state=RANDOM_STATE).reset_index(drop=True)
some_stations.head()
some_stations = pd.concat([
    some_stations,
    pd.DataFrame([{
        'station_name': 'Warehouse', 'station_id': '1.00',
        'latitude': WAREHOUSE_COORDS[1], 'longitude': WAREHOUSE_COORDS[0]
    }])
], ignore_index=True)

In [90]:
print(some_stations.shape)
print(stations.shape)

(20, 4)
(2217, 4)


# Check for uniqueness

In [91]:
id_counts = some_stations['station_id'].value_counts()
nonid_counts = id_counts[id_counts > 1]
print(f"The data is unique: {nonid_counts.empty}")


The data is unique: True


# Compute the Net-Flow Vector

In [92]:
some_stations_data = biciData[
    biciData['start_station_id'].isin(some_stations['station_id']) &
    biciData['end_station_id'].isin(some_stations['station_id'])
]
some_stations_data.shape

(35, 13)

In [93]:
some_stations_data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
14114,ABB3D6908D708D62,electric_bike,2025-11-03 07:11:49.393,2025-11-03 07:24:22.325,Pier 40 X2,SYS033,Pier 40 X2,SYS033,40.728487,-74.011693,40.728487,-74.011693,member
15663,9859813FB11FDE6A,classic_bike,2025-11-06 08:36:50.170,2025-11-06 08:49:30.955,Montague St & Clinton St,4677.06,Coffey St & Conover St,4137.10,40.694271,-73.992327,40.677236,-74.015665,casual
15776,DE4001705AEC9484,classic_bike,2025-11-07 08:06:41.041,2025-11-07 08:20:26.002,Montague St & Clinton St,4677.06,Coffey St & Conover St,4137.10,40.694271,-73.992327,40.677236,-74.015665,member
21648,125D7041AE10A9A8,electric_bike,2025-11-03 08:56:54.522,2025-11-03 08:58:05.062,E 33 St & 5 Ave,6322.01,E 33 St & 5 Ave,6322.01,40.747659,-73.984907,40.747659,-73.984907,member
36116,622882337311658D,electric_bike,2025-11-05 07:48:15.167,2025-11-05 08:02:29.371,Montague St & Clinton St,4677.06,Coffey St & Conover St,4137.10,40.694271,-73.992327,40.677236,-74.015665,member


In [94]:
"Now we want to compute how many trips occur to a node and how many occur away from a node"

# inflow to the end station
inflow_counts = some_stations_data['end_station_id'].value_counts().to_dict()

# Compute outflow from the start station
outflow_counts = some_stations_data['start_station_id'].value_counts().to_dict()

station_netFlow = {}
for station in some_stations['station_id']:
    inflow = inflow_counts.get(station, 0)
    outflow = outflow_counts.get(station, 0)
    net_flow = inflow - outflow
    station_netFlow[station] = net_flow

# convert into a dataframe
some_stations['net_flow'] = some_stations['station_id'].map(station_netFlow)
print(sum(some_stations['net_flow']))
# Positive Values indicate Source Nodes, Negative Values indicate Sink Nodes



0


In [95]:
some_stations.head(N_STATIONS+1)

,station_name,station_id,latitude,longitude,net_flow
0,Pier 40 X2,SYS033,40.728487,-74.011693,0
1,Stagg St & Union Ave,5117.05,40.708771,-73.950953,0
2,35 Ave & 10 St,6806.06,40.763155,-73.939855,-1
3,36 Ave & 10 St,6737.03,40.761438,-73.941126,-1
4,30 Ave & 96 St,6772.11,40.760860,-73.873080,0
5,14 St & 5 Ave,3771.06,40.666287,-73.988951,0
6,Woodside Ave & 55 St,6276.02,40.746790,-73.908930,-7
7,47 Ave & Skillman Ave,6237.01,40.743752,-73.941765,7
8,Montague St & Clinton St,4677.06,40.694271,-73.992327,2
9,44 St & 48 Ave,6058.08,40.739414,-73.921320,0


# Check for uniqueness

In [96]:
id_counts = some_stations['station_id'].value_counts()
nonid_counts = id_counts[id_counts > 1]
print(f"The data is unique: {nonid_counts.empty}")


The data is unique: True


# Build Distance Matrix between the stations

To do this we can use the project-osrm API to find the distances and durations between points

In [97]:
URL = "https://router.project-osrm.org/table/v1/driving/{coords}?annotations=duration,distance"

""" By inputting a list of coordinates of the type (long, lat), 
    it returns the distance and duration matrices between those coordinates. 
    We just need to split the long lat by a comma and each coordinate by a semicolon."""

coords = ";".join(f"{lon},{lat}" for lon, lat in zip(some_stations['longitude'], some_stations['latitude']))

# given the 
# get the response 
response = pd.read_json(URL.format(coords=coords))
# save the response in a json file
response.to_json("../processed_data/distance_duration_matrix.json", indent=4)


In [98]:
# Get the distance and duration dictionaries from the json file
with open("../processed_data/distance_duration_matrix.json", "r") as f:
    data = json.load(f)
distance_dict = data['distances']
duration_dict = data['durations']

distance_matrix = np.empty((N_STATIONS+1, N_STATIONS+1))
duration_matrix = np.empty((N_STATIONS+1, N_STATIONS+1))

for r in range(N_STATIONS+1):
    distance_matrix[r, :] = distance_dict[str(r)]
    duration_matrix[r, :] = duration_dict[str(r)]

station_name_to_row = {station_name: idx for idx, station_name in enumerate(stations['station_name'])}

distance_matrix /= 1000  # convert to kilometers
distance_matrix = distance_matrix.astype(np.float32)
print("Distance matrix (in kilometers):")
print(distance_matrix[-5:,-5:])

duration_matrix /= 60  # convert to seconds
duration_matrix = duration_matrix.astype(np.float32)
print("Duration matrix (in seconds):")
print(duration_matrix[-5:,-5:])

Distance matrix (in kilometers):
[[ 0.     21.8429 17.439  17.0604 11.5964]
 [24.1129  0.      5.4701  6.3997 11.5431]
 [18.885   5.7533  0.     10.3293  6.7565]
 [16.4177  7.7202 10.6717  0.      6.4597]
 [15.2759 10.4209  7.4293  6.5113  0.    ]]
Duration matrix (in seconds):
[[ 0.       29.028334 27.215    20.868334 19.07    ]
 [28.92      0.       10.341666  9.875    16.178333]
 [28.663334 10.291667  0.       16.275    11.858334]
 [22.425    11.138333 15.858334  0.       10.328333]
 [21.381666 15.06     12.63     10.508333  0.      ]]


# Final Review of Data

In [99]:
pd.DataFrame(distance_matrix).to_csv(output_dirs[0], index=False, header=False)
pd.DataFrame(duration_matrix).to_csv(output_dirs[1], index=False, header=False)
pd.DataFrame(some_stations).to_csv(output_dirs[2], index=False)

# Check for uniqueness

In [100]:
# Change the station_id column to a stripped string
some_stations['station_id'] = some_stations['station_id'].astype(str).str.strip()

id_counts = some_stations['station_id'].value_counts()
nonid_counts = id_counts[id_counts > 1]
print(some_stations[some_stations['station_id'].isin(nonid_counts.index)])
print(f"The data is unique: {nonid_counts.empty}")


Empty DataFrame
Columns: [station_name, station_id, latitude, longitude, net_flow]
Index: []
The data is unique: True


In [101]:
print(some_stations.iloc[38])
print(some_stations.iloc[47])
# print(some_stations.iloc[99])

IndexError: single positional indexer is out-of-bounds